In [1]:
using Clp, Gurobi, JuMP, NamedArrays, CSV, DataFrames

__Note:__ this model doesn't fully include the gas costs, etc. it can be modified to include that. Just pushed the code to show a working version

## Problem Definition

In [2]:
# Has Acadia to Zion w/ entrance fee, lodging, est length of stay, gas price
dat = DataFrame(CSV.File("./park_data.csv")) # park data
;

In [3]:
days = [0] # for madison
append!(days, dat[!, "est. length of stay"])

park_costs = [3.5] # for madison (only fuel)
append!(park_costs, dat[!, "lodging"] .* dat[!, "est. length of stay"] .+ dat[!, "entrance fee"])

49-element Vector{Float64}:
    3.5
 1030.0
  148.0
  200.0
  610.0
  132.0
  110.0
  215.0
  152.0
  238.0
    ⋮
  229.0
  160.0
  300.0
  158.0
   97.0
  130.0
  825.0
 2495.0
  227.0

In [4]:
# Has park data with UW-Madison as the first row and column (data to the top left instead of bottom)
cost = CSV.File("./data.csv", header=0) |> Tables.matrix

49×49 Matrix{Int64}:
      0   77996   69258   36345   76598  …   42740   65128  102214   84374
  77996       0  138027  113521  139198     119913  142298  171030  153161
  69258  137296       0   39821   51232      37477   29739   39038   18374
  36345  113316   40178       0   62808       4345   28589   67436   52567
  76598  138936   51233   62612       0      61963   74680   74341   58639
  81427   99414  128487  111282   94702  …  117167  136424  155547  135938
  67103  135188   10609   37714   50300      35370   35679   46408   25743
  80968  148979   14893   50297   60157      46589   31086   30273    6430
  70097  138138    1777   40663   50488      38319   30580   39880   19216
  73523  141508    7422   44034   56805      41689   28951   34377   12675
      ⋮                                  ⋱       ⋮                  
 111043  179200   45114   78918   70541  …   75210   59509    9324   31016
  45960   45944  102709   81158   91470      87550  109935  138508  117843
  43222  1

## Use the Adapative Subtour Elimination Method
This helps eliminate subtours by sort of formulating this similar to the TSP version but with only the set of nodes selected by the first optimal solution (that may include subtours). This limits subtour constraints by not forcing the model to visit all points

In [5]:
# Code from https://nbviewer.org/github/asmith28/cs524-su22/blob/main/TravelingSalesmanProblem.ipynb
# modified to suit our needs

# HELPER FUNCTION: returns the cycle containing the park START.
function getSubtour(x,start)
    subtour = [start]
    while true
        j = subtour[end]
        for k in parks
            if x[k,j] == 1
                push!(subtour,k)
                break
            end
        end
        if subtour[end] == start
            break
        end
    end
    return subtour
end

# HELPER FUNCTION: returns a list of all cycles
function getAllSubtours(x, specific_parks)
    nodesRemaining = specific_parks
    subtours = []
    while length(nodesRemaining) > 0
        subtour = getSubtour(x, nodesRemaining[1])
        push!(subtours, subtour)
        nodesRemaining = setdiff(nodesRemaining, subtour)
    end
    return subtours
end
;



In [6]:
parks = [1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49]

m = Model(Gurobi.Optimizer)
set_optimizer_attribute(m, "OutputFlag", 0)

@variable(m, x[parks, parks], Bin) # flow from one park to another

@constraint(m, c1[i in parks], sum(x[i,:]) - sum(x[:,i]) == 0)

# enforce starting points
@constraint(m, sum(x[1, :]) == 1)
@constraint(m, sum(x[:, 1]) == 1)

@constraint(m, self_loop[i in parks], x[i, i] == 0) # no self-loops
@constraint(m, sum(days[i]*sum(x[i,:]) for i in parks) >= 7) # Min days spent
@constraint(m, sum(days[i]*sum(x[i,:]) for i in parks) <= 7) # Max days spent

@objective(m, Min, sum(cost[i, j] * x[i, j] for i in parks, j in parks))
;

Set parameter Username
Academic license - for non-commercial use only - expires 2023-07-13


The code below uses adaptive subtour elimination to add constraints that prevent subtours from forming. I've also modified the `getAllSubtours` helper function to only filter through the unique nodes provided by the first-run solution that may contain subtours. As Julia thinks this is optimal, we will add constraints to not have subtours for those. After this, Julia will iteratively run this code to determine the optimal solution with the no-subtour constraints applied

In [7]:
# Code from https://nbviewer.org/github/asmith28/cs524-su22/blob/main/TravelingSalesmanProblem.ipynb

sols = []
opt_sol = []

# We'll run the heuristic 30 times and hope we get an optimal solution
for iters = 1:30
    optimize!(m)
    # total  length of current tour
    println("Tour length: ", objective_value(m))
    xx = value.(x) # save solution
    push!(sols, xx) # save solution

    nodes = []
    for i in parks
        for j in parks
            if Int(value(x[i, j])) == 1
                append!(nodes, [i, j])
            end
        end
    end

    subtours = getAllSubtours(xx, unique!(nodes))  # get all the subtours
    display(subtours)
    sleep(1)
    # get length of the subtour list
    len = length(subtours)
    if len == 1
        # solution is just a single tour!
        println("SOLVED!")

        # Calculate total days spent
        tot_days = 0
        for day in subtours[1]
            tot_days += days[day]
        end
        println("Total days spent: ", tot_days)

        # Show the optimal route
        for i in parks
            for j in parks
                if Int(value(x[i,j])) == 1
                    println([i,j])
                end
            end
        end
        opt_sol = subtours[1]
        println(opt_sol)

        break
    else
        for subtour in subtours
            L = length(subtour)
            # add constraints that cut off each subtour in the list (add two for each subtour)
            @constraint(m, sum(x[subtour[k+1], subtour[k]] for k = 1:L-1) <= L - 2)
            @constraint(m, sum(x[subtour[k], subtour[k+1]] for k = 1:L-1) <= L - 2)
        end
    end
end

2-element Vector{Any}:
 [1, 26, 1]
 [8, 49, 8]

Tour length: 33865.0


3-element Vector{Any}:
 [1, 17, 1]
 [4, 46, 4]
 [11, 24, 11]

Tour length: 50948.0


2-element Vector{Any}:
 [1, 26, 14, 1]
 [28, 41, 28]

...

Tour length: 89671.0


1-element Vector{Any}:
 [1, 26, 23, 30, 17, 1]

Tour length: 89963.0


SOLVED!
Total days spent: 7


[1, 17]
[17, 30]
[23, 26]
[26, 1]
[30, 23]
[1, 26, 23, 30, 17, 1]


In [8]:
objective_value(m)

89963.0

In [9]:
println("Arcs in optimal solution are:")
path_opt = []
for i in parks
    for j in parks
        if Int(value(x[i,j])) == 1
            append!(path_opt, [[i,j]])
        end
    end
end

# This describes the __actual__ arcs of the solution rather than
# the nodes as shown when a subtour of length 1 is found
# NOTE: the way the solution is drawn will be the same as it's a closed loop
# but the optimal objective value depends on the arcs
path_opt

Arcs in optimal solution are:


5-element Vector{Any}:
 [1, 17]
 [17, 30]
 [23, 26]
 [26, 1]
 [30, 23]

## Optimal Solution

In [10]:
# Redefine park / location names to be easily readable
park_names = ["UW-Madison",
    "Acadia National Park",
    "Arches National Park",
    "Badlands National Park",
    "Big Bend National Park",
    "Biscayne National Park",
    "Black Canyon of the Gunnison National Park",
    "Bryce Canyon National Park",
    "Canyonlands National Park",
    "Capitol Reef National Park",
    "Carlsbad Caverns National Park",
    "Congaree National Park",
    "Crater Lake National Park",
    "Cuyahoga Valley National Park",
    "Death Valley National Park",
    "Everglades National Park",
    "Gateway Arch National Park",
    "Glacier National Park",
    "Grand Canyon National Park",
    "Grand Teton National Park",
    "Great Basin National Park",
    "Great Sand Dunes National Park",
    "Great Smoky Mountains National Park",
    "Guadalupe Mountains National Park",
    "Hot Springs National Park",
    "Indiana Dunes National Park",
    "Joshua Tree National Park",
    "Kings Canyon National Park",
    "Lassen Volcanic National Park",
    "Mammoth Cave National Park",
    "Mesa Verde National Park",
    "Mount Rainier National Park",
    "New River Gorge National Park",
    "North Cascades National Park",
    "Olympic National Park",
    "Petrified Forest National Park",
    "Pinnacles National Park",
    "Redwood National Park",
    "Rocky Mountain National Park",
    "Saguaro National Park",
    "Sequoia National Park",
    "Shenandoah National Park",
    "Theodore Roosevelt National Park",
    "Voyageurs National Park",
    "White Sands National Park",
    "Wind Cave National Park",
    "Yellowstone National Park",
    "Yosemite National Park",
    "Zion National Park"]
;

# Describes optimal solution!
opt_sol_names = []
for path in path_opt
    append!(opt_sol_names, [park_names[path[1]]])
end
append!(opt_sol_names, ["UW-Madison"]) # last node

opt_sol_names # shows flow of closed-loop solution

6-element Vector{Any}:
 "UW-Madison"
 "Gateway Arch National Park"
 "Great Smoky Mountains National Park"
 "Indiana Dunes National Park"
 "Mammoth Cave National Park"
 "UW-Madison"